# Overview of Solvers II

## Goal
- Get an overview of various solvers in Ocean
- Understand the main purpose of each solver
- Get familiar with some basic solver parameters
- Get familiar with embedding

## Problem
To focus on sampler, we are going to create a simple BQM problem that we will solve using different solvers.

In [ ]:
from dimod import AdjVectorBQM
bqm = AdjVectorBQM('SPIN')
bqm.add_variable(0, -1)
bqm.add_variable(1, -1)
bqm.add_variable(4, -1)
bqm.add_variable(5, -1)
bqm.add_interaction(0, 4, 1.0)
bqm.add_interaction(0, 5, 1.0)
bqm.add_interaction(1, 4, 1.0)
bqm.add_interaction(1, 5, 1.0)
bqm.add_interaction(0, 1, 1.0)
bqm.add_interaction(4, 5, 1.0)

import networkx as nx

nx.draw(to_networkx_graph(bqm))

## ExactSolver
- Mainly for debugging purposes
- Can solve problems with up to 20 variables (or more) depending on the system

In [ ]:
from dimod import ExactSolver

solver = ExactSolver()
response = solver.sample(bqm)
print(response.truncate(10))

## DWaveSampler
- The main interface to use the quantum annealing processor
- It can select different quantum processors including the most recent Advantage system
- It can solve optimization problems casted as an Ising Hamiltonian
- The solver API allow submitting more abstract problems forms such as QUBO and BQM

In [ ]:
from dwave.system import DWaveSampler

sampler = DWaveSampler()

response = sampler.sample(
    bqm, num_reads=10,
    annealing_time=10,
    auto_scale=False,
    answer_mode='raw'
    )
print(response)

## What happened?
- The graph of qubit connectivity is not fully connected. 
- Some qubit-qubit interactions do not exist
- If the problem graph has interactions that don't exist, we cannot solve that problem directly
- What should we do?

## EmbeddingComposite
- As mentioned above, the graph of a problem and the processor may not be compatible
- One may use a chain of qubits that are strongly connected to act as a single qubit
- This is a complicated process that requires an hour long lecture
- The `EmbeddingComposite` abstracts away all the complications

In [ ]:
from dwave.system import EmbeddingComposite
from dwave.system import DWaveSampler

sampler = EmbeddingComposite(DWaveSampler())

response = sampler.sample(
    bqm, num_reads=10,
    annealing_time=10,
    auto_scale=True,
    answer_mode='raw',
    return_embedding=True
    )
print(response)

In [ ]:
print(response.info.get('embedding_context').get('embedding'))

## DWaveCliqueSampler
- More abstractions
- If you have a dense or fully connected problem, it's much better to use a clique sampler than heuristic embedding because heuristic embedding may not find efficient embeddings (e.g. it may find embeddings with larger chains and uneven chain length).
- Even more abstraction $\rightarrow$ `DWaveCliqueSampler` is a standalone sampler

In [ ]:
from dwave.system import DWaveCliqueSampler

sampler = DWaveCliqueSampler()

response = sampler.sample(
    bqm, num_reads=10,
    annealing_time=10,
    answer_mode='raw'
    )
print(response)

## LeapHybridSampler
- The most flexible solver 
- Can solve large, dense problems efficiently using classical and quantum resources
- 20,000 variable fully connected (~200M biases)
- 1 million variables with at most 200M biases
- Only one parameter - time limit (the minimum time limit is chosen by default)

In [ ]:
from dwave.system import LeapHybridSampler

sampler = LeapHybridSampler()
print(sampler.properties)

response = sampler.sample(
    bqm, time_limit=10,
    )
print(response)